In [4]:
import pandas as pd 
import numpy as np 

from statsmodels.tsa import arima_model
from statsmodels.graphics.tsaplots import acf, pacf, plot_acf, plot_pacf

import matplotlib.pyplot as plt 
import seaborn as sns 

sns.set()

In [7]:
df = pd.read_csv('./ui_timeseries.csv', sep=',')

df.drop(columns=['Unnamed: 0'], inplace=True)

df.head()

,Series,Periode,Permintaan (unit)
0,1,04/07/2008,19210
1,2,04/14/2008,18650
2,3,04/21/2008,14309
3,4,04/28/2008,25226
4,5,05/5/2008,24377


In [8]:
df.shape

(96, 3)

In [9]:
df.dtypes

Series                int64
Periode              object
Permintaan (unit)     int64
dtype: object

In [10]:
df['Periode'] = df['Periode'].astype('datetime64[ns]')

df.head()

,Series,Periode,Permintaan (unit)
0,1,2008-04-07,19210
1,2,2008-04-14,18650
2,3,2008-04-21,14309
3,4,2008-04-28,25226
4,5,2008-05-05,24377


In [11]:
df = df.set_index('Periode')

df.head()

,Series,Permintaan (unit)
Periode,,
2008-04-07,1,19210
2008-04-14,2,18650
2008-04-21,3,14309
2008-04-28,4,25226
2008-05-05,5,24377
